[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb)

# Scaling to Many Routes and Using Pinecone

Semantic router can be used with many hundreds, thousands, or even more routes. At very large scales it can be useful to use a vector database to store and search though your route vector space. Although we do not demonstrate _very large_ scale in this notebook, we will demonstrate more routes than usual and we will also see how to use the `PineconeIndex` for potential scalability and route persistence beyond our local machines.

## Installing the Library

In [ ]:
!pip install -qU \
    "semantic-router[local, pinecone]==0.0.22" \
    datasets==2.17.0

## Downloading Routes

In [1]:
from datasets import load_dataset

data = load_dataset("aurelio-ai/generic-routes", split="train")
data

/Users/jamesbriggs/opt/anaconda3/envs/decision-layer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since aurelio-ai/generic-routes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/jamesbriggs/.cache/huggingface/datasets/aurelio-ai___generic-routes/default/0.0.0/5ed6ce316bb803dc716232e6c5f0eb1c7400e24d (last modified on Sun Feb 18 15:49:32 2024).


Dataset({
    features: ['name', 'utterances', 'description', 'function_schema', 'llm', 'score_threshold'],
    num_rows: 50
})

Each row in this dataset is a single route:

In [2]:
data[0]

{'name': 'politics',
 'utterances': ["isn't politics the best thing ever",
  "why don't you tell me about your political opinions",
  "don't you just love the presidentdon't you just hate the president",
  "they're going to destroy this country!",
  'they will save the country!'],
 'description': None,
 'function_schema': None,
 'llm': None,
 'score_threshold': 0.82}

We transform these into `Route` objects like so:

In [3]:
from semantic_router import Route

routes = [Route(**data[i]) for i in range(len(data))]
routes[0]

Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schema=None, llm=None, score_threshold=0.82)

Next we initialize an `encoder`. We will use a simple `HuggingFaceEncoder`, we can also use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`.

In [4]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

Now we initialize our `PineconeIndex`, all it requires is a [Pinecone API key](https://app.pinecone.io) (you do need to be using Pinecone Serverless).

In [5]:
import os
from getpass import getpass
from semantic_router.index import PineconeIndex

os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

index = PineconeIndex()

2024-02-18 17:11:50 WARNING semantic_router.utils.logger Index could not be initialized.


In [8]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-02-18 17:12:21 INFO semantic_router.utils.logger local


We run the solely static routes layer:

In [9]:
rl("how's the weather today?").name

'chitchat'

_If you see a warning about no classification being found, wait a moment and run the above cell again._

## Loading Index From Previous Initialization

Because we're using Pinecone our route index can now persist / be access from different locations by simply connecting to the pre-existing index, by default this index uses the identifier `"semantic-router--index"` — this is the index we'll be loading here, but we can change the name via the `index_name` parameter if preferred.

First, let's delete our old route layer, `index`, and `routes`.

In [10]:
del rl, index, routes

Let's load our index first. As mentioned, `"index"` is the default index name, so we don't need to specify this parameter — but we do so below for demonstrative purposes.

In [11]:
index = PineconeIndex(index_name="index")

We load the pre-existing routes from this index like so:

In [12]:
index.get_routes()

[('fitness_tips', 'suggest a workout routine'),
 ('daily_inspiration', 'give me an inspirational quote'),
 ('creative_writing_and_literature', 'how can I improve my writing skills?'),
 ('chitchat', "let's go to the chippy"),
 ('astronomy_and_space_exploration',
  'what are some interesting facts about the universe?'),
 ('chitchat', 'the weather is horrendous'),
 ('cloud_computing', 'AWS vs Azure vs Google Cloud'),
 ('chitchat', 'how are things going?'),
 ('educational_facts', 'tell me an interesting fact'),
 ('chitchat', "how's the weather today?"),
 ('ethical_considerations_in_tech', 'ethical hacking and its importance'),
 ('compliments', 'say something nice about me'),
 ('food_and_recipes', "what's your favorite food?"),
 ('interview_preparation', 'common programming interview questions'),
 ('gaming_and_esports', 'what are the popular games right now?'),
 ('frameworks_and_libraries',
  "what's the difference between React and Angular?"),
 ('environmental_awareness', 'how can I be mor

We will transform these into a dictionary format that we can use to initialize our `Route` objects.

In [13]:
routes_dict = {}
for route, utterance in index.get_routes():
    if route not in routes_dict:
        routes_dict[route] = []
    routes_dict[route].append(utterance)

In [14]:
routes_dict

{'jokes': ['tell me a joke', 'make me laugh', 'know any good jokes?'],
 'career_advice': ['suggest some career development tips',
  'what are the emerging career fields?',
  'how can I improve my resume?'],
 'environmental_awareness': ['tell me about sustainability practices',
  'how can I be more eco-friendly?',
  'what are some ways to save the planet?'],
 'data_structures_and_algorithms': ['algorithms every developer should know',
  'basic data structures for beginners',
  'complexity analysis of algorithms'],
 'chitchat': ['lovely weather today',
  "how's the weather today?",
  'how are things going?',
  'the weather is horrendous',
  "let's go to the chippy"],
 'daily_inspiration': ['share something uplifting',
  'give me an inspirational quote',
  'I need some inspiration for today'],
 'career_advice_in_tech': ['how to build a portfolio for software development',
  'navigating career growth in tech',
  'tips for landing your first tech job'],
 'cloud_computing': ['best practices 

Now we transform these into a list of `Route` objects.

In [15]:
routes = [
    Route(name=route, utterances=utterances)
    for route, utterances in routes_dict.items()
]
routes[0]

Route(name='jokes', utterances=['tell me a joke', 'make me laugh', 'know any good jokes?'], description=None, function_schema=None, llm=None, score_threshold=None)

Now we reinitialize our `RouteLayer`:

In [16]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-02-18 17:16:19 INFO semantic_router.utils.logger local


And test it again:

In [18]:
rl("say something to make me laugh").name

'jokes'

In [21]:
rl("tell me something amusing").name

'jokes'

In [27]:
rl("it's raining cats and dogs today").name

'chitchat'

Perfect, our routes loaded from our `PineconeIndex` are working as expected! As mentioned, we can use the `PineconeIndex` for persistance and high scale use-cases, for example where we might have hundreds of thousands of utterances, or even millions.

---